# Extras not in the final notebooks (for now)

### Fit curve to the max curve seen above to calculate threshold for lower genes, curve elbow, etc. 

In [ ]:
# make bins and take maximum of each bin to get (x,y) points to fit sigmoid curve to
df = pl_expression_vs_detection(adata, cell_type='Plasma', return_df=True)
df['percent_detected'] = df['percent_detected']/100
df_bins = df.groupby(pd.cut(df['log1p(means)'], np.arange(0, max(df['log1p(means)']), 0.05))).max() 
#TODO issue here that the log1p(means) value is also always the max, as in the very top of the bin instead of the median for example? 
df_bins = df_bins.dropna()
# fit function here
xdata = df_bins['log1p(means)'].values
ydata = df_bins['percent_detected']
for bin, value in ydata.items():
    if (bin.right > 1.25) and (value < 0.7):
        ydata.loc[bin] = 0.999
ydata = ydata.values
df_bins.tail()

In [ ]:
# fit and plot splines
tck = splrep(xdata, ydata, s=0.0009, k=3)
tck_s = splrep(xdata, ydata, s=len(xdata))

# derivatives of spline function
yders = interpolate.spalde(xdata, tck)
yders_df = pd.DataFrame(yders, columns=['y', 'dy', 'd2y', 'd3y'], index= xdata)
yders_df['d3y_diff'] = yders_df['d3y'].diff().fillna(0)
# identify any changes in direction of the third derivative
infls = yders_df[yders_df['d3y_diff'] != 0].index
infls

In [ ]:
# plot data vs fitted spline with inflection points
fig, ax2 = plt.subplots(1,1)
ax2.scatter(data=df, x='log1p(means)', y='percent_detected', alpha=0.5, color='orange', s=2.5)
ax2.plot(xdata, BSpline(*tck)(xdata), label='s=0.001')
#ax2.plot(xdata, BSpline(*tck_s)(xdata), label=f's={len(xdata)}')
ax2.set_title("Spline fit to expression vs. fraction detected")
ax2.set_xlabel("log1p(means)")
ax2.set_ylabel("fraction detected")
ax2.vlines(infls, 0, 1.05, color='red', linestyles='dashed', label='turning points')
ax2.legend()
plt.show()

In [ ]:
# second derivative of spline function
yders = interpolate.spalde(xdata, tck)
plt.figure()
for i in range(4):
   plt.plot(xdata, [d[i] for d in yders], '--', label=f"{i} derivative")
plt.legend()
plt.title('All derivatives of B-spline')
plt.show()

In [ ]:
# function to plot relationship between total counts mean and percent detected value
# x: cell type specific total counts mean, y: cell type specific percent detected value 

def pl_tc_vs_pd(adata, ax, method='mean', return_df = False, layer='log_norm'):

    # calculate mean total counts per cell type
    subset_df = adata.to_df(layer='log_norm')
    means = adata.obs.groupby(['celltypist_cell_label']).mean()
    new_df = pd.DataFrame(data=means['total_counts']) #df with cell type as index and mean 'total_counts' as column

    temp = {}
    # calculate percentage of cells (per cell type) where each gene is detected (average)
    for cell_type in adata.obs['celltypist_cell_label'].unique():
        subset = subset_df[adata.obs['celltypist_cell_label'] == cell_type]
        nonzero_detected = pd.DataFrame(np.count_nonzero(subset, axis=0) / len(subset)*100, columns=[cell_type], index=subset.columns)
        if method=='median':
            temp[cell_type] = nonzero_detected[cell_type].median()
        else:
            temp[cell_type] = nonzero_detected[cell_type].mean()

    new_df['percent_detected'] = new_df.index.map(temp)
    new_df = new_df.sort_values(['total_counts'], ascending=True)
    if return_df:
        return new_df
    
    # plot new_df: x=total_counts, y=percent_detected
    for row, column in new_df.iterrows():
        ax.scatter(column['total_counts'], column['percent_detected'], c=np.random.rand(1, 3), label=row) 
        ax.annotate(row, (column['total_counts'], column['percent_detected']), (column['total_counts']+100, column['percent_detected']+0.1), fontsize=6.5)

    ax.set_title("Mean total counts vs. mean percent detected, per cell type")
    ax.set_xlabel("Mean total counts")
    ax.set_ylabel(str(method + " percent non-zero"))
    
    return ax

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10,6))
ax1 = pl_tc_vs_pd(adata, ax=ax1, method='median')
plt.show()
# TODO look online for cell size estimates for each cell type and compare to total counts? 

## Histograms! 

In [ ]:
# function to plot histogram for given adata layer
def clean_data(adata, layer_name, threshold = 99.75, remove_zeros = False):
    df = adata.to_df(layer=layer_name)
    data = df.to_numpy().flatten()
    
    cutoff = np.percentile(df.values, threshold)
    if remove_zeros:
        data = data[data != 0]
    data = data[data <= cutoff]
    
    return data
    

def plot_hist(adata, ax, layer_name, bin_num=1000): # higher bin numbers made zeros in tc_norm_log disappear? 
    data = clean_data(adata, layer_name)
    
    hist, edges = np.histogram(data, bins=bin_num)

    ax.bar(edges[:-1], hist, width = max(edges)/bin_num, color='#0504aa', align='edge')
    ax.set_title(layer_name)
    ax.set_xlim(min(edges), np.percentile(data, 99.75))
    ax.set_ylim(0, np.percentile(hist, 99.75))
    ax.grid(axis='y', alpha=0.75)
    ax.set_xlabel('Gene Expression')
    ax.set_ylabel('Frequency')
    ax.vlines(np.percentile(data, 97.5), ymin=0, ymax=max(hist), linestyle="--", color="r", label='97.5%')
        
    ax.legend()
    
    return ax

In [ ]:
# plot histograms
fig, (ax2, ax3, ax4) = plt.subplots(1,3, figsize=(16,6), sharey=True)
ax2 = plot_hist(adata[adata.obs['celltypist_cell_label_coarse'] == 'Plasma'], ax=ax2, layer_name='log_norm', bin_num=500)
ax2.set_title("Plasma")
ax3 = plot_hist(adata[adata.obs['celltypist_cell_label_coarse'] == 'T Cell'], ax=ax3, layer_name='log_norm', bin_num=500)
ax3.set_title("T Cell")
ax4 = plot_hist(adata[adata.obs['celltypist_cell_label'] == 'TA'], ax=ax4, layer_name='log_norm', bin_num=500)
ax4.set_title("TA")
plt.show()

In [ ]:
# function to calculate the percent change between two layers (e.g. log_norm and magic)
def calculate_percent_change(adata, layer1, layer2, type='value'):
    df1 = adata.to_df(layer=layer1)
    df2 = adata.to_df(layer=layer2)
    
    nonzero_detected = pd.DataFrame(np.count_nonzero(df1, axis=0), columns=['df1_nonzero'], index=df1.columns)
    nonzero_detected['df2_nonzero'] = np.count_nonzero(df2, axis=0)
    if type == 'value':
        # calculate percent change of all values (including 0s)
        nonzero_detected['percent_change'] = (nonzero_detected['df2_nonzero'] - nonzero_detected['df1_nonzero']) / nonzero_detected['df1_nonzero'] * 100
    elif type=='nonzero':
        # calculate percentage of the number of values that changed from 0 to non-zero
        nonzero_detected['percent_change'] = (nonzero_detected['df2_nonzero'] - nonzero_detected['df1_nonzero']) / len(df1) * 100
    nonzero_detected['percent_change'] = nonzero_detected['percent_change'].fillna(0)
    return nonzero_detected

# function to plot the percent change between two layers (e.g. log_norm and magic)
def plot_percent_change(adata, layer1, layer2, cell_type, type='value', col='log1p(means)'):
    title=str(GOI)
    if cell_type != None:
        title = str("Overall expression in " + cell_type + " cells")
        if cell_type in adata.obs['celltypist_cell_label_coarse'].unique():
            adata = adata[adata.obs['celltypist_cell_label_coarse'] == cell_type]
        elif cell_type in adata.obs['celltypist_cell_label'].unique():
            adata = adata[adata.obs['celltypist_cell_label'] == cell_type]
        else:
            print("Cell type not found. Please check spelling.")
            return
    else:
        title = "All cell types"
        
    df = make_df(adata, col, layer=layer1)

    changes = calculate_percent_change(adata, layer1, layer2, type=type)
    df = df.join(changes, how='left').sort_values(['gene_num'], ascending=True)
        
    # plot mean expression of a gene (x) vs. percentage of cells where this gene is detected (y)
    ax = sns.scatterplot(data=df, x=col, y='percent_change', hue='expr_class', linewidth=0)
    ax.set_title(str(title + ": Expression vs. Change in Detection"))
    ax.legend(title='Expression Class', loc='lower left', bbox_to_anchor=(1, 0))
    plt.show()
    return ax